In [1]:
using ColoringNames

In [2]:
runnum = "4"
function mklogdir(name)
    logdir = joinpath(pwd(), "logs", name * runnum)
    mkpath(logdir)
    logdir
end

mklogdir (generic function with 1 method)

In [3]:
function qualitative_demo(mdl, do_oov=true)
    plot_query(mdl,  "brownish green") |> display
    plot_query(mdl,  "brown") |> display
    plot_query(mdl,  "brownish") |> display
    plot_query(mdl,  "greenish") |> display
    plot_query(mdl,  "greenish brown") |> display
    plot_query(mdl,  "green")|> display
    do_oov && plot_query(mdl,  "ish")|> display
end

qualitative_demo(mdl::TermToColorDistributionEmpirical) = qualitative_demo(mdl, false)

qualitative_demo (generic function with 3 methods)

In [4]:
many_names = load_color_nameset()
word_embs, vocab, enc = load_text_embeddings(keep_words=many_names)
cldata = load_munroe_data(dev_as_train=false, dev_as_test=true, encoding_ = enc)

ColoringNames.ColorDatasets{MLLabelUtils.LabelEnc.NativeLabels{String,2833},ColoringNames.ColorDataset{Array{Any,1},Array{Int64,2},Array{Float32,2}}}(MLLabelUtils.LabelEnc.NativeLabels{String,2833}(String["and", "of", "a", "with", "'s", "not", "-", "your", "just", "can"  …  "Feijoa", "melanzane", "minsk", "Pizazz", "feijoa", "red1", "Kilamanjaro", "karaka", "bilbao", "mabel"], Dict("Apple"=>157,"Caper"=>2160,"chocolate"=>424,"Casablanca"=>1155,"Many"=>84,"Luxor"=>1613,"star"=>103,"canyon"=>900,"bombay"=>2563,"bondi"=>2662…)), ColoringNames.ColorDataset{Array{Any,1},Array{Int64,2},Array{Float32,2}}(Any["acid green", "acid green", "acid green", "acid green", "acid green", "acid green", "acid green", "acid green", "acid green", "acid green"  …  "yuck", "yuck", "yuck", "yuck", "yuck", "yuck", "yuck", "yuck", "yuck", "yuck"], [338 338 … 2148 2148; 114 114 … 0 0; 0 0 … 0 0; 0 0 … 0 0], Float32[0.344113 0.855319 0.921569; 0.281905 0.744681 0.921569; … ; 0.376894 0.768559 0.898039; 0.179104 0.

In [5]:
sowe_unsmoothed = TermToColorDistributionSOWE(enc, word_embs; n_steps=size(cldata.train.terms_padded,1))
train!(sowe_unsmoothed, cldata, Val{false}();
        log_dir = mklogdir("sowe_unsmoothed"),
        batch_size = 829,
        min_epochs=200,
        max_epochs=30_000,
        )
@show evaluate(sowe_unsmoothed, cldata.dev)

2018-05-21 19:06:23.885020: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2


evaluate(sowe_unsmoothed, cldata.dev) = Dict(:perp=>163.035,:perp_val=>217.683,:mse_to_peak=>0.273704,:perp_sat=>309.611,:perp_hue=>64.2991)


Dict{Symbol,Float64} with 5 entries:
  :perp        => 163.035
  :perp_val    => 217.683
  :mse_to_peak => 0.273704
  :perp_sat    => 309.611
  :perp_hue    => 64.2991

In [6]:
qualitative_demo(sowe_unsmoothed)

LoadError: [91mMethodError: no method matching setcharheight(::Float64)
The applicable method may be too new: running in world age 21939, while current world is 21940.[0m
Closest candidates are:
  setcharheight(::Real) at /home/wheel/oxinabox/.julia/v0.6/GR/src/GR.jl:1077 (method too new to be called from this world context.)[39m

In [7]:
sowe_smoothed = TermToColorDistributionSOWE(enc, word_embs; n_steps=size(cldata.train.terms_padded,1))
train!(sowe_smoothed, cldata, Val{true}();
        log_dir=mklogdir("sowe_smoothed"),
        batch_size = 829,
        min_epochs=200,
        max_epochs=30_000,
        )
@show evaluate(sowe_smoothed, cldata.dev)

evaluate(sowe_smoothed, cldata.dev) = Dict(:perp=>163.694,:perp_val=>219.936,:mse_to_peak=>0.235125,:perp_sat=>308.257,:perp_hue=>64.6979)


Dict{Symbol,Float64} with 5 entries:
  :perp        => 163.694
  :perp_val    => 219.936
  :mse_to_peak => 0.235125
  :perp_sat    => 308.257
  :perp_hue    => 64.6979

In [ ]:
rnn_unsmoothed = TermToColorDistributionRNN(enc, word_embs; n_steps=size(cldata.train.terms_padded,1))
train!(rnn_unsmoothed, cldata, Val{false}();
        log_dir = mklogdir("rnn_unsmoothed"),
        batch_size = 829,
        max_epochs=30_000,
        )
@show evaluate(rnn_unsmoothed, cldata.dev)


In [ ]:
qualitative_demo(rnn_unsmoothed)

In [ ]:
rnn_smoothed = TermToColorDistributionRNN(enc, word_embs; n_steps=size(cldata.train.terms_padded,1))
train!(rnn_smoothed, cldata, Val{true}();
        log_dir = mklogdir("rnn_smoothed"),
        batch_size = 829,
        max_epochs=30_000,
        )

@show evaluate(rnn_smoothed, cldata.dev)

In [ ]:
qualitative_demo(rnn_smoothed)

In [ ]:
cnn_unsmoothed = TermToColorDistributionCNN(enc, word_embs; n_steps=size(cldata.train.terms_padded,1))
train!(cnn_unsmoothed, cldata, Val{false}();
        log_dir=mklogdir("cnn_unsmoothed"),
        batch_size = 829,
        max_epochs=30_000,
        )

@show evaluate(cnn_unsmoothed, cldata.dev)

In [ ]:
qualitative_demo(cnn_unsmoothed)

In [ ]:
cnn_smoothed = TermToColorDistributionCNN(enc, word_embs; n_steps=size(cldata.train.terms_padded,1))
train!(cnn_smoothed, cldata, Val{true}();
        log_dir=mklogdir("cnn_smoothed_rand"),
        batch_size = 829,
        max_epochs=30_000,
        )

@show evaluate(cnn_smoothed, cldata.dev)

In [ ]:
qualitative_demo(cnn_smoothed)

In [ ]:
noml_unsmoothed = TermToColorDistributionEmpirical(256)
train!(noml_unsmoothed, cldata, Val{false}())
@show evaluate(noml_unsmoothed, cldata.dev)

In [ ]:
qualitative_demo(noml_unsmoothed)

In [ ]:
noml_smoothed = TermToColorDistributionEmpirical(256)
train!(noml_smoothed, cldata, Val{true}())
@show evaluate(noml_smoothed, cldata.dev)

In [ ]:
qualitative_demo(noml_smoothed)